This notebook listed a few reusable objects extracted from the full NLP call transcript use case
-----------

Make sure pip is up-to-date
-----------

In [ ]:
#pip install --upgrade pip

Prepare libraries
--

In [ ]:
pip install -U scikit-learn scipy matplotlib

In [ ]:
pip install textblob

In [ ]:
pip install wordcloud

In [ ]:
pip install gensim

In [ ]:
import pandas as pd 
import re
import string
import pickle
import sys

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords

import collections
from collections import Counter

from wordcloud import WordCloud

import matplotlib.pyplot as plt

from textblob import TextBlob

from gensim import matutils, models
import scipy.sparse

Import data and set the structure
---

In [ ]:
df = pd.read_csv('//file_direction/filename', sep='|')
df.shape

In [ ]:
df

In [ ]:
#Group the transcripts ("content") per Call ID ("call_id")
df=df.groupby(['call_id' ])['content'].apply(' '.join).reset_index() 
df.shape

In [ ]:
df

Cleaning 1 - RE for general grooming
---

In [ ]:
# Apply a first round of text cleaning techniques
#import re
#import string

def re_clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

r1 = lambda x: re_clean_text(x)

In [ ]:
#Calling re_clean_text function
data_clean = pd.DataFrame(df.content.apply(r1))
data_clean

#  Cleaning 2 - stemming
-----------------------------------
Steps to stem a document

1. take the doc as the input
2. read line by line
3. tokenize the line
4. stem the words
5. output the stemmed words

Use Cases:
sentimental analysis
document clustering
information retrieval



In [ ]:
#import nltk
#nltk.download('punkt')

In [ ]:
#A second round of cleanning technique - stemming
#from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.stem import PorterStemmer

st=PorterStemmer()

def stem_text(text):
    token_words=word_tokenize(text)
    stem_text=[]
    for word in token_words:
        stem_text.append(st.stem(word))
        stem_text.append(" ")
    return "".join(stem_text)

s1=lambda x: stem_text(x)

In [ ]:
# Let's take a look at the updated text (a longer run time, 30~ mins)
data_clean = pd.DataFrame(data_clean.content.apply(s1))
data_clean

# Cleaning 3 - removing Stop Word
_______________________________________________

In [ ]:
#import nltk
#from nltk.tokenize import word_tokenize 

#nltk.download('stopwords')
#from nltk.corpus import stopwords

# set of stop words
stop_words = set(stopwords.words('english')) 

In [ ]:
def rm_stopword(text):
    # tokens of words  
    token_words=word_tokenize(text)
    # define variable
    filtered_sentence = [] 
    
    for word in token_words:
        if word not in stop_words:
            filtered_sentence.append(word)
            filtered_sentence.append(" ")
    return "".join(filtered_sentence)

rsw=lambda x: rm_stopword(x)

In [ ]:
#Apply stop word removal
data_clean = pd.DataFrame(data_clean.content.apply(rsw))
data_clean 

In [ ]:
# To add the call id back to the dataframe
data_clean['call_id'] = df['call_id']
data_clean


Pickle the file to a local drive
-----------------------------

In [ ]:
#import pickle

In [ ]:
# Make a new directory (stage_Data/) to hold the intermediate pickle files
!mkdir stage_data

In [ ]:
# Pickle files for later use (to save it physically to local drive)
data_clean.to_pickle("stage_data/cust_data_clean.pkl")


#Read from pickle file to verify
#pd.read_pickle('stage_data/data_clean3.pkl')

Clearning 4 - finding Common Words
------------------

In [ ]:
#import collections

Define the functin of top Common Word Count per document (call) within the file
-------

In [ ]:
#import re
#from collections import Counter

def word_counts_line (text):
    counts = []
    #to convert the line of words into a list of words
    text = re.findall(r'\w+', text)
    
    #counts the number each time a word appears and get the top 30 common words
    counts = Counter(text).most_common(30) 
    return counts

cd = lambda x: word_counts_line(x)
 

In [ ]:
#Apply stop word removal
pd.DataFrame(data_clean.content.apply(cd))

Define the functin of top Common Word Count per file  
-------

In [ ]:
# Look at the 100 most common top words --> add them to the stop word list
#from collections import Counter

# To pull out 100 top words from each call
top100_word_cnt = Counter(" ".join(data_clean["content"]).split()).most_common(100)
top100_word_cnt

In [ ]:
add_stop_words = [word for word, count in top100_word_cnt if count > 500]
add_stop_words

Clearning 5 - Apply additional stop words
------------------

In [ ]:
len(stop_words)

In [ ]:
len(add_stop_words)

In [ ]:
#Apply additional stop word removal
stop_words = stop_words.union(add_stop_words)

data_clean = pd.DataFrame(data_clean.content.apply(rsw))
data_clean

In [ ]:
#adding the call id back to the datagrame
data_clean['call_id'] = df['call_id'] 
data_clean

In [ ]:
#Save the data to pickle file
data_clean.to_pickle("stage_data/data_clean.pkl")

Run Word Clouds
---------

In [ ]:
#To make some word clouds
#from wordcloud import WordCloud

wc = WordCloud(stopwords=stop_words, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

Extract Noun function
-----------------------

In [ ]:
#import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

In [ ]:
# Let's create a function to pull out nouns from a string of text
#from nltk import word_tokenize, pos_tag

def nouns(text):
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

Prepare stop words
---------

In [ ]:
# iterate through the csv file 
from wordcloud import WordCloud, STOPWORDS 

comment_words = '' 
stopwords = set(STOPWORDS) 

stopwords = stopwords.union(add_stop_words)
stopwords = stopwords.union(stop_words)

In [ ]:
for val in data_clean.content: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value      
    comment_words += " ".join(val.split() )+" "
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
#Apply Noun extraction function to failed si
text_nouns = pd.DataFrame(data_clean.content.apply(nouns))
text_nouns

In [ ]:
#Adding call id back to the dataframe 
text_nouns['call_id'] = data_clean['call_id']
text_nouns

In [ ]:
text_nouns.to_pickle("stage_data/text_nouns.pkl")

Sentiment Analysis
---

In [ ]:
#import sys
#from textblob import TextBlob

In [ ]:
# Create lambda functions to find the polarity and subjectivity per call
p = lambda x: TextBlob(x).sentiment.polarity
s= lambda x: TextBlob(x).sentiment.subjectivity

data_clean['polarity'] = data_clean['content'].apply(p)
data_clean['subjectivity'] = data_clean['content'].apply(s)
data_clean

In [ ]:
data_clean.describe()

In [ ]:
data_clean.polarity[data_clean.polarity< 0].sum()

Topic Modeling - Attempt #1 (All Text)
-----------------------------------------

In [ ]:
df=pd.read_pickle("stage_data/data_clean.pkl")

In [ ]:
#import pandas as pd
#from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(df.content)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
print(df )

In [ ]:
#pip install gensim

In [ ]:
# Import the necessary modules for LDA with gensim
#from gensim import matutils, models
#import scipy.sparse

In [ ]:
# One of the required inputs is a term-document matrix
tdm = df.transpose()
tdm.head()

In [ ]:
#Transforming the term-document matrix into corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
#Further format prep for Gensim
vec
id2word = dict((v, k) for k, v in vec.vocabulary_.items())

In [ ]:
#To define two other parameters - first try 2 topics and 10 passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

In [ ]:
#Then try 30 topics and 10 passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=30, passes=10)
lda.print_topics()

In [ ]:
#Then try 100 topics and 10 passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=100, passes=10)
lda.print_topics()

Topic Modeling - Attempt #2 (Nouns Only)
------------------------------------------

In [ ]:
df=pd.read_pickle("stage_data/text_nouns.pkl")
df

In [ ]:
#import pandas as pd
#from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(df.content)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
print(df )

In [ ]:
tdm = df.transpose()
tdm.head()

In [ ]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
vec
id2word = dict((v, k) for k, v in vec.vocabulary_.items())

In [ ]:
#Try 30 topics and 100 passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=30, passes=100)
lda.print_topics()

Topic Modeling - potentially attemps may be including adjectives also, topic modeling per each lauguage, etc.
--------------------------------------------------

Clean up the memory (misc.)
------

In [ ]:
df.info(memory_usage='deep')

In [ ]:
import gc

In [ ]:
%whos DataFrame

In [ ]:
del [[df]]
gc.collect()
df=pd.DataFrame()
 